In [1]:
using Gen
using PyCall
using ProgressBars
pushfirst!(PyVector(pyimport("sys")."path"), "");


In [5]:
SIR = pyimport("sir")
model = SIR.SIR(1000, 10)
S,I,R  = model.project()
I[end]

35.707810173235266

In [11]:
projection_days = 90; 
N = 1300000000; #1.3 Billion population of India



@gen function sample_SIR()
    
    #Sample date of introduction
    intro = @trace(poisson(10), :intro) #Expected introduction 10 days from now
    days = projection_days - intro
    
    log_r0 = @trace(normal(log(2.6), 0.5), :log_r0)
    r0 = exp(log_r0)
    
    inf_period = @trace(normal(4.5, 0.5), :inf_period)
    
    model = SIR.SIR(N,days, r0 = r0, inf_period = inf_period)
    S,I,R = model.project()
    return S,I,R
end



([1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.3e9, 1.29999e9, 1.29999e9, 1.29999e9, 1.29999e9, 1.29998e9, 1.29997e9, 1.29997e9, 1.29996e9, 1.29994e9, 1.29992e9, 1.29989e9, 1.29986e9, 1.29981e9, 1.29974e9, 1.29966e9, 1.29954e9, 1.29938e9, 1.29918e9, 1.2989e9, 1.29853e9, 1.29803e9, 1.29736e9, 1.29648e9, 1.29529e9, 1.29371e9, 1.2916e9, 1.28879e9, 1.28506e9, 1.28012e9, 1.27358e9, 1.26497e9, 1.25369e9, 1.23901e9, 1.22009e9, 1.19595e9, 1.16561e9, 1.12815e9, 1.08288e9, 1.02958e9, 9.68682e8, 9.0141e8, 8.29732e8, 7.56154e8, 6.8335e8, 6.13758e8, 5.49264e8, 4.91059e8, 4.39654e8, 3.95013e8, 3.56728e8, 3.2418e8], [1.0, 1.33804, 1.79035, 2.39556, 3.20535, 4.28889, 5.7387, 7.67861, 10.2743, 13.7474, 18.3945, 24.6126, 32.9327, 44.0652, 58.961, 78.8921, 105.561, 141.244, 188.991, 252.877, 338.359, 452.738, 605.781, 810.558, 1084.56, 1451.18, 1941.73, 2598.11

In [11]:
poisson(10)

11

In [16]:

log(2)

0.6931471805599453